**SQL_BASICS**


In [1]:
# --- Step 1: Install required library ---
import sqlite3
import pandas as pd

# --- Step 2: Create in-memory SQLite database ---
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# --- Step 3: Create sample tables from Mavenmovies.sql ---
# Note: SQLite doesn't support ENUM, SET, AUTO_INCREMENT, DELIMITER, etc.
# We'll simplify schema definitions to be compatible with SQLite.

cursor.executescript("""
DROP TABLE IF EXISTS actor;
DROP TABLE IF EXISTS film;
DROP TABLE IF EXISTS film_actor;
DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS rental;
DROP TABLE IF EXISTS payment;
DROP TABLE IF EXISTS language;

CREATE TABLE actor (
    actor_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL
);

CREATE TABLE film (
    film_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    release_year INT,
    language_id INT,
    rental_duration INT,
    rental_rate REAL,
    length INT,
    replacement_cost REAL,
    rating TEXT
);

CREATE TABLE film_actor (
    actor_id INT,
    film_id INT,
    PRIMARY KEY (actor_id, film_id)
);

CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT,
    active INTEGER DEFAULT 1
);

CREATE TABLE rental (
    rental_id INTEGER PRIMARY KEY,
    rental_date TEXT,
    inventory_id INT,
    customer_id INT,
    return_date TEXT,
    staff_id INT
);

CREATE TABLE payment (
    payment_id INTEGER PRIMARY KEY,
    customer_id INT,
    staff_id INT,
    rental_id INT,
    amount REAL,
    payment_date TEXT
);

CREATE TABLE language (
    language_id INTEGER PRIMARY KEY,
    name TEXT
);
""")

# --- Step 4: Insert some sample data ---
cursor.executemany("INSERT INTO actor VALUES (?, ?, ?)", [
    (1, "PENELOPE", "GUINESS"),
    (2, "NICK", "WAHLBERG"),
    (3, "ED", "CHASE"),
])

cursor.executemany("INSERT INTO language VALUES (?, ?)", [
    (1, "English"),
    (2, "French")
])

cursor.executemany("INSERT INTO film VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", [
    (1, "ACADEMY DINOSAUR", "Epic drama", 2006, 1, 6, 0.99, 86, 20.99, "PG"),
    (2, "ACE GOLDFINGER", "Spy thriller", 2006, 1, 3, 4.99, 120, 15.99, "G"),
    (3, "ADAPTATION HOLES", "Comedy", 2006, 2, 7, 2.99, 90, 25.99, "PG-13")
])

cursor.executemany("INSERT INTO film_actor VALUES (?, ?)", [
    (1, 1), (2, 1), (3, 2)
])

cursor.executemany("INSERT INTO customer VALUES (?, ?, ?, ?, ?)", [
    (1, "MARY", "SMITH", "mary@example.com", 1),
    (2, "JOHN", "DOE", "john@example.com", 1),
    (3, "ALICE", "BROWN", "alice@example.com", 0)
])

cursor.executemany("INSERT INTO rental VALUES (?, ?, ?, ?, ?, ?)", [
    (1, "2023-01-01", 1, 1, "2023-01-05", 1),
    (2, "2023-02-10", 2, 2, "2023-02-12", 1)
])

cursor.executemany("INSERT INTO payment VALUES (?, ?, ?, ?, ?, ?)", [
    (1, 1, 1, 1, 5.99, "2023-01-05"),
    (2, 2, 1, 2, 2.99, "2023-02-12")
])

conn.commit()

# --- Step 5: Helper to run queries ---
def run_query(query):
    return pd.read_sql_query(query, conn)

# --- Step 6: Example Queries from your assignment ---

# 1) List all actors
print("Actors Table:")
display(run_query("SELECT * FROM actor;"))

# 2) List all films with rental_duration > 5
print("Films with rental_duration > 5:")
display(run_query("SELECT title, rental_duration FROM film WHERE rental_duration > 5;"))

# 3) Total rentals
print("Total rentals:")
display(run_query("SELECT COUNT(*) AS total_rentals FROM rental;"))

# 4) Average rental rate per language
print("Average rental rate per language:")
display(run_query("""
SELECT l.name, AVG(f.rental_rate) AS avg_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.name;
"""))


Actors Table:


,actor_id,first_name,last_name
0,1,PENELOPE,GUINESS
1,2,NICK,WAHLBERG
2,3,ED,CHASE


Films with rental_duration > 5:


,title,rental_duration
0,ACADEMY DINOSAUR,6
1,ADAPTATION HOLES,7


Total rentals:


,total_rentals
0,2


Average rental rate per language:


,name,avg_rate
0,English,2.99
1,French,2.99


In [ ]:
import sqlite3
import pandas as pd

# --- Setup DB again ---
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# --- Schema ---
cursor.executescript("""
DROP TABLE IF EXISTS actor;
DROP TABLE IF EXISTS film;
DROP TABLE IF EXISTS film_actor;
DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS rental;
DROP TABLE IF EXISTS payment;
DROP TABLE IF EXISTS language;
DROP TABLE IF EXISTS inventory;
DROP TABLE IF EXISTS address;
DROP TABLE IF EXISTS city;
DROP TABLE IF EXISTS store;
DROP TABLE IF EXISTS category;
DROP TABLE IF EXISTS film_category;

CREATE TABLE actor (
    actor_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL
);

CREATE TABLE film (
    film_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    release_year INT,
    language_id INT,
    rental_duration INT,
    rental_rate REAL,
    length INT,
    replacement_cost REAL,
    rating TEXT
);

CREATE TABLE film_actor (
    actor_id INT,
    film_id INT,
    PRIMARY KEY (actor_id, film_id)
);

CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT,
    address_id INT,
    active INTEGER DEFAULT 1
);

CREATE TABLE rental (
    rental_id INTEGER PRIMARY KEY,
    rental_date TEXT,
    inventory_id INT,
    customer_id INT,
    return_date TEXT,
    staff_id INT
);

CREATE TABLE payment (
    payment_id INTEGER PRIMARY KEY,
    customer_id INT,
    staff_id INT,
    rental_id INT,
    amount REAL,
    payment_date TEXT
);

CREATE TABLE language (
    language_id INTEGER PRIMARY KEY,
    name TEXT
);

CREATE TABLE inventory (
    inventory_id INTEGER PRIMARY KEY,
    film_id INT,
    store_id INT
);

CREATE TABLE store (
    store_id INTEGER PRIMARY KEY,
    manager_staff_id INT,
    address_id INT
);

CREATE TABLE address (
    address_id INTEGER PRIMARY KEY,
    address TEXT,
    city_id INT
);

CREATE TABLE city (
    city_id INTEGER PRIMARY KEY,
    city TEXT
);

CREATE TABLE category (
    category_id INTEGER PRIMARY KEY,
    name TEXT
);

CREATE TABLE film_category (
    film_id INT,
    category_id INT,
    PRIMARY KEY (film_id, category_id)
);
""")

# --- Insert sample data ---
cursor.executemany("INSERT INTO actor VALUES (?, ?, ?)", [
    (1, "PENELOPE", "GUINESS"),
    (2, "NICK", "WAHLBERG"),
    (3, "ED", "CHASE"),
])

cursor.executemany("INSERT INTO language VALUES (?, ?)", [
    (1, "English"),
    (2, "French")
])

cursor.executemany("INSERT INTO film VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", [
    (1, "ACADEMY DINOSAUR", "Epic drama", 2006, 1, 6, 0.99, 86, 20.99, "PG"),
    (2, "GONE WITH THE WIND", "Classic drama", 1939, 1, 5, 3.99, 120, 19.99, "G"),
    (3, "ADAPTATION HOLES", "Comedy", 2006, 2, 7, 2.99, 90, 25.99, "PG-13")
])

cursor.executemany("INSERT INTO film_actor VALUES (?, ?)", [
    (1, 2), (2, 2), (3, 1)
])

cursor.executemany("INSERT INTO customer VALUES (?, ?, ?, ?, ?, ?)", [
    (1, "MARY", "SMITH", "mary@example.com", 1, 1),
    (2, "JOHN", "DOE", "john@example.com", 2, 1),
    (3, "ALICE", "BROWN", "alice@example.com", 3, 0)
])

cursor.executemany("INSERT INTO city VALUES (?, ?)", [
    (1, "London"),
    (2, "Paris")
])

cursor.executemany("INSERT INTO address VALUES (?, ?, ?)", [
    (1, "10 Downing St", 1),
    (2, "221B Baker St", 1),
    (3, "Champs Elysees", 2)
])

cursor.executemany("INSERT INTO store VALUES (?, ?, ?)", [
    (1, 101, 1),
    (2, 102, 2)
])

cursor.executemany("INSERT INTO inventory VALUES (?, ?, ?)", [
    (1, 1, 1),
    (2, 2, 1),
    (3, 2, 2),
    (4, 3, 1)
])

cursor.executemany("INSERT INTO rental VALUES (?, ?, ?, ?, ?, ?)", [
    (1, "2023-01-01", 1, 1, "2023-01-05", 1),
    (2, "2023-02-10", 2, 2, "2023-02-12", 1),
    (3, "2023-03-05", 3, 1, "2023-03-08", 2)
])

cursor.executemany("INSERT INTO payment VALUES (?, ?, ?, ?, ?, ?)", [
    (1, 1, 1, 1, 5.99, "2023-01-05"),
    (2, 2, 1, 2, 2.99, "2023-02-12"),
    (3, 1, 2, 3, 4.50, "2023-03-08")
])

cursor.executemany("INSERT INTO category VALUES (?, ?)", [
    (1, "Drama"),
    (2, "Comedy")
])

cursor.executemany("INSERT INTO film_category VALUES (?, ?)", [
    (1, 1),
    (2, 1),
    (3, 2)
])

conn.commit()

def run_query(q):
    return pd.read_sql_query(q, conn)

# --------------------------
# 🔹 Joins
# --------------------------

# Q9: Movie title + customer name
print("Q9 - Movies rented with customer names:")
display(run_query("""
SELECT f.title, c.first_name, c.last_name
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer c ON r.customer_id = c.customer_id;
"""))

# Q10: Actors in "Gone with the Wind"
print("Q10 - Actors in Gone with the Wind:")
display(run_query("""
SELECT a.first_name, a.last_name
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.title = 'GONE WITH THE WIND';
"""))

# Q11: Customer spending
print("Q11 - Total spending per customer:")
display(run_query("""
SELECT c.first_name, c.last_name, SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id;
"""))

# Q12: Movies rented by customers in London
print("Q12 - Movies rented by customers in London:")
display(run_query("""
SELECT c.first_name, c.last_name, f.title
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE ci.city = 'London'
GROUP BY c.customer_id, f.title;
"""))

# Q13: Top 5 rented movies
print("Q13 - Top 5 rented movies:")
display(run_query("""
SELECT f.title, COUNT(r.rental_id) AS times_rented
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id
ORDER BY times_rented DESC
LIMIT 5;
"""))

# Q14: Customers who rented from both stores
print("Q14 - Customers with rentals from both stores:")
display(run_query("""
SELECT c.first_name, c.last_name
FROM customer c
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY c.customer_id
HAVING COUNT(DISTINCT i.store_id) = 2;
"""))

# --------------------------
# 🔹 Window Functions
# --------------------------

# 1. Rank customers by total spent
print("WF1 - Rank customers by spending:")
display(run_query("""
SELECT c.first_name, c.last_name, SUM(p.amount) AS total_spent,
       RANK() OVER (ORDER BY SUM(p.amount) DESC) AS rank
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id;
"""))

# 2. Cumulative revenue by film
print("WF2 - Cumulative revenue per film:")
display(run_query("""
SELECT f.title, p.payment_date, SUM(p.amount) OVER (
       PARTITION BY f.film_id ORDER BY p.payment_date
       ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
) AS cumulative_revenue
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id;
"""))

# 3. Avg rental duration by film length
print("WF3 - Avg rental duration by film length:")
display(run_query("""
SELECT f.title, f.length, AVG(f.rental_duration) OVER (PARTITION BY f.length) AS avg_duration
FROM film f;
"""))

# 4. Top 3 films per category by rental count
print("WF4 - Top 3 films per category by rental count:")
display(run_query("""
SELECT c.name AS category, f.title, COUNT(r.rental_id) AS rental_count,
       RANK() OVER (PARTITION BY c.name ORDER BY COUNT(r.rental_id) DESC) AS rnk
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.name, f.title
HAVING rnk <= 3;
"""))

# 6. Monthly revenue trend
print("WF6 - Monthly revenue trend:")
display(run_query("""
SELECT strftime('%Y-%m', p.payment_date) AS month,
       SUM(p.amount) AS revenue,
       SUM(SUM(p.amount)) OVER (ORDER BY strftime('%Y-%m', p.payment_date)) AS cumulative_revenue
FROM payment p
GROUP BY month;
"""))

# 7. Customers in top 20% spending
print("WF7 - Top 20% spenders:")
display(run_query("""
WITH spending AS (
  SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_spent
  FROM customer c
  JOIN payment p ON c.customer_id = p.customer_id
  GROUP BY c.customer_id
)
SELECT * FROM spending
WHERE total_spent >= (SELECT PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY total_spent) FROM spending);
"""))

# 8. Running total of rentals per category
print("WF8 - Running total of rentals per category:")
display(run_query("""
SELECT c.name AS category, f.title, COUNT(r.rental_id) AS rentals,
       SUM(COUNT(r.rental_id)) OVER (PARTITION BY c.name ORDER BY f.title) AS running_total
FROM film f
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY c.name, f.title;
"""))

# 9. Films rented less than category avg
print("WF9 - Films rented less than category avg:")
display(run_query("""
WITH film_rentals AS (
  SELECT f.film_id, f.title, c.name AS category, COUNT(r.rental_id) AS cnt
  FROM film f
  JOIN film_category fc ON f.film_id = fc.film_id
  JOIN category c ON fc.category_id = c.category_id
  JOIN inventory i ON f.film_id = i.film_id
  JOIN rental r ON i.inventory_id = r.inventory_id
  GROUP BY f.film_id
)
SELECT * FROM film_rentals fr
WHERE fr.cnt < (SELECT AVG(cnt) FROM film_rentals WHERE category = fr.category);
"""))

# 10. Top 5 months by revenue
print("WF10 - Top 5 revenue months:")
display(run_query("""
SELECT strftime('%Y-%m', p.payment_date) AS month, SUM(p.amount) AS revenue
FROM payment p
GROUP BY month
ORDER BY revenue DESC
LIMIT 5;
"""))
